
# Build a basic ME model

We will try to build an ME model from the NC_000913.2 Genbank file, the iJO1366 M model, and the complex reconstruction from iJL1650-ME

In [ ]:
# python imports
import re
import json
from os.path import join

# third party imports
import pandas
import escher
import cobra.test

# ecoli me
import ecolime
from ecolime.flat_files import *
from ecolime.ecoli_k12 import *

from minime import *
from minime.util import building
from minime.util.building import add_ribosomes, add_RNA_polymerase
from minime.util.building import add_complex_stoichiometry_data, add_transcription_reaction
from minime.util.building import add_complex_modification_data
from minime.util.building import add_metabolic_reactions
from minime.solve.algorithms import binary_search, fva, solve_at_growth_rate
from minime.solve.symbolic import compile_expressions

## Colton TODOs 
- 1) look into all of the free modifications in the original ME and make sure none of thes reactions are blocked
- 2) Add demands for cell wall biosynthesis/dna
- 3) Look through check list on Github

In [ ]:
from minime.util.mass import compute_RNA_mass

## Begin by loading metabolites and build Metabolic reactions

In [ ]:
me = MEmodel('iJO1366-ME')
m_model = ecolime.get_m_model()
# some of the "metabolites" in iJO1366 "M" model are actually complexes. We pass those in
# so they get created as complexes, not metabolites.
complex_list = []
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("CPLX"))
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("EG"))
complex_list.extend(i.id for i in m_model.metabolites if "-MONOMER" in i.id)
complex_list.extend(i.id for i in m_model.metabolites if "-CPLX" in i.id)
complex_list.extend(i.id for i in m_model.metabolites if "_mod_" in i.id)
# temp fix
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("Isc"))
complex_list = set(complex_list)
building.add_m_model_content(me, m_model, complex_metabolite_ids=complex_list)

# if the bounds of this metabolite aren't open, model uses wrong reactions
me.reactions.EX_pqq_e.lower_bound = -1000

Sometimes multiple entities can perform the same role. To prevent a combinatorial explosion of possibilities, we can create  "generic" version, where any of those entities can fill in.

In [ ]:
for generic, components in get_generics().items():
    GenericData(generic, me, components).create_reactions()

## Build ribosome and RNA Polymerase

In [ ]:
gb_filename = join(ecoli_files_dir,'NC_000913.2.gb')                                                                                    
TU_df = pandas.read_csv(join(ecoli_files_dir,'TUs_from_ecocyc.txt'), delimiter="\t", index_col=0)

building.build_reactions_from_genbank(me, gb_filename, TU_df)

In [ ]:
add_ribosomes(me)

add_RNA_polymerase(me)

Make a dummy reactions. Add a dummy protein in as well.

In [ ]:
building.add_dummy_reactions(me, "ATG" + "TTT" * 12 + "TAT" * 12 + "ACG" * 12 + "GAT" * 12 + "AGT" * 12 + "TAA")

## Associate the tRNA synthetases

The tRNA charging reactions were automatically added when loading the genome from the genbank file. However, the charging reactions still need to be made aware of the tRNA synthetases which are responsible.

In [ ]:
with open(join(ecoli_files_dir, "amino_acid_tRNA_synthetase.json"), "rb") as infile:
    aa_synthetase_dict = json.load(infile)
for data in me.tRNA_data:
    data.synthetase = str(aa_synthetase_dict[data.amino_acid])

### Add in complex Formation with modifications

In [ ]:
# ME_complex_dict is a dict of {'complex_id': [{'bnum' : count}]}
rna_components = {"b3123"} # component id should have RNA_ instead of protein_
ME_complex_dict = get_complex_to_bnum_dict(rna_components)
# some entries in the file need to be renamed.
# Colton 7/8/15 made changes directly to flat file
#renames = {"MnmE_": "b3706", "MnmG_": "b3741", "YheM_": "b3344", "YheL_": "b3343", "YheN_": "b3345"}
add_complex_stoichiometry_data(me, ME_complex_dict)

In [ ]:
rxn_dict = get_reaction_matrix_dict()
met_list = []
for met_stoich in rxn_dict.values():
    for met in met_stoich:
        met_list.append(met.replace('_c',''))
met_list.pop('EG10544-MONOMER_mod_palmitate')
modification_dict = get_protein_modification_dict(met_list,  generic=True)
add_complex_modification_data(me, modification_dict)

In [ ]:
# two different reactiond can add a lipoate modification.
# We create a separate ModificationData for each one
lipo = me.modification_data.get_by_id("mod_lipo_c")
alt_lipo = ModificationData("mod_lipo_c_alt", me)
#alt_lipo.stoichiometry = lipo.stoichiometry

lipo.stoichiometry = {"lipoamp_c": -1, "amp_c": 1}
lipo.enzyme = 'EG11796-MONOMER'
lipo.keff = 65.

alt_lipo.stoichiometry = {'EG50003-MONOMER_mod_pan4p_mod_lipo':-1,
                          'EG50003-MONOMER_mod_pan4p':1,
                          'h_c':-1,}
alt_lipo.enzyme = 'EG11591-MONOMER'
alt_lipo.keff = 65.

for cplx_data in lipo.get_complex_data():
    alt_cplx_data = ComplexData(cplx_data.id + "alt", me)
    alt_cplx_data.complex_id = cplx_data.complex_id
    alt_cplx_data.translocation = cplx_data.translocation
    alt_cplx_data.chaperones = cplx_data.chaperones
    alt_cplx_data.modifications = cplx_data.modifications
    alt_cplx_data.modifications[alt_lipo.id] = \
        alt_cplx_data.modifications.pop(lipo.id)

In [ ]:
# chaperones
# todo bmocogdp_c mods
# todo FE-S modifications
mod_catalysts = {'CPLX0-1762':'G6712-MONOMER', # FE-S modification
                 'TMAOREDUCTI-CPLX':'EG12195-MONOMER', #bmocogdp
                 'DIMESULFREDUCT-CPLX':'G6849-MONOMER', #bmocogdp
                 'NITRATREDUCTA-CPLX':'NARJ-MONOMER', #bmocogdp
                 'NITRATREDUCTZ-CPLX':'NARW-MONOMER', #bmocogdp
                 'NAP-CPLX':'NAPD-MONOMER', #bmocogdp
                 'NAPAB-CPLX_NAPC-MONOMER':'NAPD-MONOMER'} #bmocogdp

## TODO NEXT complete Iron-Sulfur cluster modification reactions. Apply rRNA/tRNA fraction constraint 

In [ ]:
essential_list = [
    #'4fe4s_c',
    'LI_c',
    #'2fe2s_c',
    '3fe4s_c',
    'palmitate_c'
    'NiFeCoCN2_c', 'acetyl_c',
    'RNase_m5','RNase_m16','RNase_m23'] # RNAses are gaps in model

for met_id in essential_list:
    r = cobra.Reaction("EX_" + met_id)
    me.add_reaction(r)
    r.reaction = met_id + " <=> "

In [ ]:
me.modification_data.

Build all complex formation reactions

In [ ]:
for cplx_data in me.complex_data:
    formation = cplx_data.formation
    if formation:
        formation.update()
    else:
        cplx_data.create_complex_formation()

In [ ]:
# these guys can transfer assembled iron sulfur clusters to the various enzymes
fes_transfer = {"erpA": "CPLX0-7617", "iscA": "IscA_tetra", "sufA": "CPLX0-7824"}

suf_cplx = ComplexData("sufBC2DES_pathway_complex", me)
suf_cplx.stoichiometry = {"CPLX0-1341": 1, "CPLX0-246_CPLX0-1342_mod_pydx5p": 1}
suf_cplx.create_complex_formation()
    
isc_cplx = ComplexData("iscUS_cyaY_pathway_complex", me) # could add chaperones into here
isc_cplx.stoichiometry = {"IscU": 1, "IscS_mod_2:pydx5p": 1, "EG11653-MONOMER": 1}
isc_cplx.create_complex_formation()

generic_fes_transfer = GenericData("generic_fes_transfer", me, ['CPLX0-7617', 'CPLX0-7824', 'IscA_tetra'])
generic_fes_transfer.create_reactions()

#building.add_and_update_metabolic_reaction(me, "isc_2fe2s_formation", suf_cplx.id, False, update=True)
#building.add_and_update_metabolic_reaction(me, "suf_2fe2s_formation", isc_cplx.id, False, update=True)
#building.add_and_update_metabolic_reaction(me, "isc_4fe4s_formation", suf_cplx.id, False, update=True)
#building.add_and_update_metabolic_reaction(me, "suf_4fe4s_formation", isc_cplx.id, False, update=True)

In [ ]:
me.modification_data.mod_2fe2s_c.enzyme = generic_fes_transfer.id
me.modification_data.mod_2fe2s_c.keff = 65.
me.modification_data.mod_4fe4s_c.enzyme = generic_fes_transfer.id
me.modification_data.mod_4fe4s_c.keff = 65.

Crutch reactions for mets that are blocked. TODO remove

## Associate Complexes with Reactions

In [ ]:
# associate reaction id with the old ME complex id (including modifications)
rxnToModCplxDict = get_reaction_to_complex(generic=True)

In [ ]:
rxn_info = get_reaction_info_frame()
for reaction_data in me.stoichiometric_data:
    add_metabolic_reactions(me, reaction_data, rxnToModCplxDict, rxn_info, update=True,
                            create_new=True)

Rebuild transcription and translation to use tRNA (now that tRNA synthetase complexes are in the model

In [ ]:
for r in me.reactions:
    if isinstance(r, tRNAChargingReaction):
        r.update()
for r in me.reactions:
    if isinstance(r, TranslationReaction):
        r.update()
    if isinstance(r, TranscriptionReaction):
        r.update()
    if isinstance(r, MetabolicReaction):
        r.update()

In [ ]:
met_list

## Add in translocation

## Make RNA splicing machinery

In [ ]:
generic_RNase = GenericData("generic_RNase", me,
                            {ecoli_k12.replace_divalent(i) for i in generic_RNase_list})
generic_RNase.create_reactions()

rRNA_machinery = ComplexData("rRNA_containing_excision_machinery", me)
rRNA_machinery.stoichiometry = {ecoli_k12.replace_divalent(i): 1 for i in ecoli_k12.rRNA_containing}
rRNA_machinery.create_complex_formation()
rRNA_mod = ModificationData("rRNA_containing_excision", me)
rRNA_mod.stoichiometry = {'h2o_c': -1, 'h_c': 1}
rRNA_mod.enzyme = rRNA_machinery.id

monocistronic_machinery = ComplexData("monocistronic_excision_machinery", me)
monocistronic_machinery.stoichiometry = {ecoli_k12.replace_divalent(i): 1 for i in ecoli_k12.monocistronic}
monocistronic_machinery.create_complex_formation()
monocistronic_mod = ModificationData("monocistronic_excision", me)
monocistronic_mod.stoichiometry = {'h2o_c': -1, 'h_c': 1}
monocistronic_mod.enzyme = monocistronic_machinery.id

polycistronic_machinery = ComplexData("polycistronic_excision_machinery", me)
polycistronic_machinery.stoichiometry = {ecoli_k12.replace_divalent(i): 1 for i in ecoli_k12.polycistronic_wout_rRNA}
polycistronic_machinery.create_complex_formation()
polycistronic_mod = ModificationData("polycistronic_excision", me)
polycistronic_mod.stoichiometry = {'h2o_c': -1, 'h_c': 1}
polycistronic_mod.enzyme = polycistronic_machinery.id

for t in me.transcription_data:
    n_excised = sum(t.excised_bases.values())
    n_cuts = len(t.RNA_products) * 2
    if n_excised == 0 or n_cuts == 0:
        continue
    RNA_types = list(t.RNA_types)
    n_tRNA = RNA_types.count("tRNA")
    
    if "rRNA" in set(RNA_types):
        t.modifications[rRNA_mod.id] = n_cuts
    elif n_tRNA == 1:
        t.modifications[monocistronic_mod.id] = n_cuts
    elif n_tRNA > 1:
        t.modifications[polycistronic_mod.id] = n_cuts
    else:  # only applies to rnpB
        t.modifications[monocistronic_mod.id] = n_cuts

## Remove unused protein and mRNA

This should make the model solve faster

This gives the total number of genes included

In [ ]:
len(me.reactions.query("transcription"))

In [ ]:
len(me.metabolites.query('RNA_b'))

## Attempt to set keffs

In [ ]:
divalent_list = divalent_list
monovalent_list = ['_mod_k','_mod_na1']
from pickle import load
with open("test_keffs.pickle", "rb") as infile:
    old_keffs = load(infile)
keffs = {}

for keff, value in old_keffs.items():
    for i in divalent_list: 
        keff = keff.replace(i, 'generic_divalent')
    for i in monovalent_list: 
        keff = keff.replace(i, '_mod_generic_monovalent')
    keffs[keff] = value
    
for r in me.reactions:
    if isinstance(r, MetabolicReaction) and r.complex_data is None:
        continue
    if isinstance(r, MetabolicReaction) and r.complex_data.id != "CPLX_dummy":
        met_rxn = r
        key = met_rxn.id.replace("-", "_DASH_").replace("__", "_DASH_").replace(":","_COLON_")
        #key = met_rxn.id
        key = "keff_" + key.replace("_FWD_", "_").replace("_REV_", "_")

        matches = [i for i in keffs if key in i]
        # get the direction
        if met_rxn.reverse:
            matches = [i for i in matches if i.endswith("_reverse_priming_keff")]
        else:
            matches = [i for i in matches if i.endswith("_forward_priming_keff")]
        if len(matches) == 1:
            met_rxn.keff = keffs[matches[0]]
        elif len(matches) > 0:
            if len(matches) == len([i for i in matches if key + "_mod_"]):
                met_rxn.keff = keffs[matches[0]]
            else:
                print key, len(matches)
        else:  # len(matches) == 0
            print "no keff found for", key

In [ ]:
for r in me.reactions:
    if hasattr(r, "update"):
        r.update()

In [ ]:
for c in me.complex_data:
    try:
        c.create_complex_formation()
        print c
    except:
        pass

In [ ]:
for c_d in me.complex_data:
    c = c_d.complex
    if len(c.reactions) == 1:
        list(c.reactions)[0].delete(remove_orphans=True)

In [ ]:
for p in me.metabolites.query("protein"):
    delete = True
    for rxn in p._reaction:
        if p in rxn.reactants:
            delete = False       
    if delete:
        print p
        while len(p._reaction) > 0:
            list(p._reaction)[0].delete(remove_orphans=True)
for m in me.metabolites.query("RNA"):
    delete = True
    for rxn in m._reaction:
        if m in rxn.reactants and not rxn.id.startswith('DM_'):
            delete = False       
    if delete:
        while len(m._reaction) > 0:
            list(m._reaction)[0].delete(remove_orphans=True)

In [ ]:
for p in me.metabolites.query("RNA_b0001"):
    print p, p.reactions

In [ ]:
demands = {'murein5px4p_p': 0.01389, 'kdo2lipid4_e': 0.01945, 'pe160_c': 0.01786,
           'pe160_p }

## Solve

In [ ]:
me.reactions.dummy_reaction_FWD_CPLX_dummy.objective_coefficient = 1.

In [ ]:
# Turn off reactions that throw off results as in iOL
KO_list = ['DHPTDNR','DHPTDNRN', 'SUCASPtpp','SUCFUMtpp', 'SUCMALtpp', 'SUCTARTtpp', 
           'CAT', 'FHL', 'SPODM', 'SPODMp']
for reaction in KO_list:
    a = me.reactions.query(reaction + '_')
    for rxn in a:
        rxn.upper_bound = 0
        rxn.lower_bound = 0

In [ ]:
me.reactions.EX_glc__D_e.lower_bound = -100

In [ ]:
expressions = compile_expressions(me)

In [ ]:
solve_at_growth_rate(me, 0.1, compiled_expressions=expressions)

In [ ]:
binary_search(me, min_mu=2, max_mu=2.1, mu_accuracy=1e-2,
              compiled_expressions=expressions)

In [ ]:
transcription = me.get_transcription_flux()

In [ ]:
{k: v for k, v in me.solution.x_dict.items() if k.startswith("DM_RNA_") and v > 1e-6}

In [ ]:
import escher
view = escher.Builder("iJO1366.Central metabolism")
view.reaction_data = me.get_metabolic_flux()
view.display_in_notebook()